In [4]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# Method 1: Using Python's urllib (recommended for cross-platform compatibility)
import urllib.request
import os

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filename = 'input.txt'

# Check if file already exists
if not os.path.exists(filename):
    print(f"Downloading {filename}...")
    urllib.request.urlretrieve(url, filename)
    print(f"Downloaded {filename} successfully!")
else:
    print(f"{filename} already exists.")

# Check file size to confirm download
if os.path.exists(filename):
    file_size = os.path.getsize(filename)
    print(f"File size: {file_size:,} bytes")

Downloaded input.txt successfully!
File size: 1,115,394 bytes
Downloaded input.txt successfully!
File size: 1,115,394 bytes


In [5]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print(f"Length of dataset in characters: {len(text):,}")

Length of dataset in characters: 1,115,394


In [7]:
print(text[:1000])  # Print the first 1000 characters to inspect the dataset

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Unique characters: {''.join(chars)}")
print(f"Vocab size: {vocab_size}")

Unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [ ]:
stoi = {ch: i for i, ch in enumerate(chars)}  # string to index
itos = {i: ch for i, ch in enumerate(chars)}  # index to string
print(f"String to index mapping: {stoi}")
print(f"Index to string mapping: {itos}")
encode = lambda s: [stoi[c] for c in s]  # 입력받은 문자열 s를 c에 해당하는 인덱스로 변환
decode = lambda l: ''.join([itos[i] for i in l])  # 입력받은 인덱스 l을 해당하는 문자열로 변환
print(f"Encode 'Hello': {encode('Hello')}")
print(decode(encode('Hello')))  # should print 'Hello'

String to index mapping: {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
Index to string mapping: {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42:

In [13]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)  # convert to tensor
print(f"Data tensor shape: {data.shape}")

Data tensor shape: torch.Size([1115394])


In [18]:
n = int(0.9 * len(data))  # first 90% for training, rest for validation
train_data = data[:n]
val_data = data[n:]
print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")

Train data shape: torch.Size([1003854])
Validation data shape: torch.Size([111540])


In [19]:
block_size = 8  # context length: how many characters to predict
train_data[:block_size +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
x = train_data[:block_size]  # input context
y = train_data[1:block_size + 1]  # targets are the same but
for i in range(block_size):
    print(x[:i+1])
    print(y[i])  # should be the next character in the sequence

tensor([18])
tensor(47)
tensor([18, 47])
tensor(56)
tensor([18, 47, 56])
tensor(57)
tensor([18, 47, 56, 57])
tensor(58)
tensor([18, 47, 56, 57, 58])
tensor(1)
tensor([18, 47, 56, 57, 58,  1])
tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15])
tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor(58)


In [26]:
torch.manual_seed(1337)  # for reproducibility
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))  # random starting indices for each sequence in the batch
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("Batch x shape:", xb.shape)
print("Batch y shape:", yb.shape)
print("Batch x:", xb)
print("Batch y:", yb)

Batch x shape: torch.Size([4, 8])
Batch y shape: torch.Size([4, 8])
Batch x: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Batch y: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)  # for reproducibility

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)  # (batch_size, block_size, vocab_size) -> (B,T,C)

        if targets is None:
            return logits, None

        B, T, C = logits.shape
        logits = logits.view(B * T, C)  # reshape to (B*T, C)
        targets = targets.view(B * T) # targets.view(-1)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # idx is (B, T) -> (B, T+1), get the prediction for the next token
            logits, loss = self(idx, None)  # forward pass
            logits = logits[:, -1, :] # take the last time step -> (B, C)
            probs = F.softmax(logits, dim=-1) # convert to probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution -> (B, 1)
            idx = torch.cat((idx,idx_next), dim=1)  # append the sampled token to the input sequence -> (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
out, loss = m(xb,yb)
print("Logits shape:", out.shape)  # should be (batch_size, block_size
print("Loss:", loss)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))  # generate 100 new tokens

Logits shape: torch.Size([32, 65])
Loss: tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [38]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [47]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    loss.backward()  # compute gradients
    optimizer.step()  # update parameters
    optimizer.zero_grad()  # reset gradients to zero

print(loss.item())

2.558009624481201


In [50]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))  # generate 100 new tokens


WA y hathe, qun s;
LO tond t, oullke CIARicousangh'dlke, y lorveloveninde searslly otharomeligse ang


The mathematical trick in self-attention

In [26]:
import torch

torch.manual_seed(1337)  # for reproducibility
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)  # random input tensor
x.shape  # should be (B, T, C)

torch.Size([4, 8, 2])

In [29]:
# we wnat x[b,t] = mean_{i <= t} x[b,i]
xbow = torch.zeros((B, T, C))  # output tensor, bow for "bag of words"
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]  # (t,C)
        xbow[b,t] = torch.mean(xprev, dim=0)  # mean over the first dimension (time)
xbow.shape  # should be (B, T, C)

torch.Size([4, 8, 2])

In [33]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)  # normalize each row to sum to 1 because we want to use it as weights
print("Weights matrix:", wei)
xbow2 = wei @ x # (T,T) @ (B,T,C) -> (B,T,C)
print(xbow2.shape)  # should be (B, T, C)
torch.allclose(xbow, xbow2,atol=1e-6)  # should be True, both methods give the same result

Weights matrix: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
torch.Size([4, 8, 2])


True

In [37]:
import torch.nn.functional as F

# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3,atol=1e-6)


True

In [45]:
import torch.nn as nn

# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # key projection
query = nn.Linear(C, head_size, bias=False) # query projection
value = nn.Linear(C, head_size, bias=False) # value projection
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)  # (B, T, 16)
out = wei @ v  # (B, T, T) @ (B, T, C) ---> (B, T, C)
#out = wei @ x  

out.shape

torch.Size([4, 8, 16])

In [47]:
k = torch.randn(B, T, head_size)  # random key tensor
q = torch.randn(B, T, head_size)  # random query tensor
wei = q @ k.transpose(-2, -1) * head_size**-0.5  # (B, T, C) @ (B, C, T) ---> (B, T, T)